# Import Dependencies

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose and Drawing utilities

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to Calculate Angle


In [3]:
def calculate_angle(a, b, c):
    """Calculates the angle between three points."""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.degrees(radians)
    angle = np.abs(angle)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# NEW Helper Function to get left knee angle if landmarks are visible 


In [4]:
def get_left_knee_angle_if_visible(landmarks, visibility_threshold=0.5):
    """
    Calculates the left knee angle if hip, knee, and ankle landmarks are sufficiently visible.
    Returns the angle in degrees, or None if landmarks are not clearly visible.
    """
    # Access landmark objects
    lm_left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    lm_left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    lm_left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

    # Check visibility
    if not (lm_left_hip.visibility > visibility_threshold and
            lm_left_knee.visibility > visibility_threshold and
            lm_left_ankle.visibility > visibility_threshold):
        return None  # Not enough reliable data

    # Get coordinates
    left_hip_coord = [lm_left_hip.x, lm_left_hip.y]
    left_knee_coord = [lm_left_knee.x, lm_left_knee.y]
    left_ankle_coord = [lm_left_ankle.x, lm_left_ankle.y]
    
    return calculate_angle(left_hip_coord, left_knee_coord, left_ankle_coord)


#  Functions to Check Squat Position


In [5]:
def is_squat_down(landmarks):
    """Checks if the person is in the 'down' position of a squat FOR COUNTING."""
    angle = get_left_knee_angle_if_visible(landmarks)
    if angle is None:
        return False
    return angle < 100  # Original threshold for counter

def is_squat_up(landmarks):
    """Checks if the person is in the 'up' position of a squat."""
    angle = get_left_knee_angle_if_visible(landmarks)
    if angle is None:
        return False
    return angle > 160


# Helper Function for "Good Depth" FEEDBACK 


In [6]:
def is_good_depth_for_feedback_message(landmarks, threshold):
    """Checks if squat depth is sufficient for 'Good Depth' feedback message."""
    angle = get_left_knee_angle_if_visible(landmarks)
    if angle is None:
        return False
    return angle < threshold

# Function for getting feedback


In [7]:
def get_feedback(landmarks):
    """Provides feedback on squat form."""
    # Check visibility of necessary landmarks for lean feedback
    if not (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility > 0.5 and
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility > 0.5 and
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility > 0.5):
        return "Tracker Obscured"

    left_hip_coord = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_shoulder_coord = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_knee_coord = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]

    LEANING_THRESHOLD = 90
    angle_hsk = calculate_angle(left_hip_coord, left_shoulder_coord, left_knee_coord)
    is_leaning_too_much = angle_hsk < LEANING_THRESHOLD

    is_generally_down_for_counter = is_squat_down(landmarks)
    is_generally_up = is_squat_up(landmarks)
    in_transition = not is_generally_down_for_counter and not is_generally_up

    GOOD_DEPTH_FEEDBACK_ANGLE_THRESHOLD = 80
    achieved_feedback_good_depth = is_good_depth_for_feedback_message(landmarks, GOOD_DEPTH_FEEDBACK_ANGLE_THRESHOLD)

    if achieved_feedback_good_depth:
        if is_leaning_too_much:
            return "Good Depth, Keep Chest Up"
        else:
            return "Good Depth"
    elif is_generally_down_for_counter:
        if is_leaning_too_much:
            return "Go Deeper & Keep Chest Up"
        else:
            return "Go Deeper"
    elif is_generally_up:
        if is_leaning_too_much:
            return "Stand Straight"
        else:
            return "Up"
    elif in_transition:
        if is_leaning_too_much:
            return "Keep Chest Up while moving"
        else:
            return "Keep Going Down"
    else:
        if is_leaning_too_much:
            return "Check Posture"
        return ""

# Process Squat Video function with Tracker Accuracy Feedback


In [8]:
def process_squat_video(input_video, output_video, show_realtime=True):
    cap = cv2.VideoCapture(input_video)
    if not cap.isOpened():
        print(f"Error: Could not open video file: {input_video}")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    if fps == 0:
        fps = 20

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    squat_counter = 0
    squat_state = "up"
    min_frame_interval = 15
    frame_count = 0
    last_counted_frame = -min_frame_interval

    # Renamed for clarity: these are the ENUMs, not the indices yet
    LANDMARKS_TO_DRAW_ENUM = [
        mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
        mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
        mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.RIGHT_KNEE,
        mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.RIGHT_ANKLE
    ]
    # Integer indices of landmarks for drawing and accuracy calculation
    KEY_LANDMARK_INDICES = [lm.value for lm in LANDMARKS_TO_DRAW_ENUM]
    VISIBILITY_THRESHOLD_FOR_ACCURACY = 0.75

    CUSTOM_SQUAT_CONNECTIONS = [
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
        (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
        (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
        (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE),
        (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE),
        (mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE),
        (mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE)
    ]
    landmark_point_spec = mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=-1, circle_radius=4)
    connection_line_spec = mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)

    GREEN_COLOR = (0, 255, 0)
    RED_COLOR = (0, 0, 255)
    WHITE_COLOR = (255, 255, 255)
    YELLOW_COLOR = (0, 255, 255)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Use a different variable name
            image_rgb.flags.writeable = False
            results = pose.process(image_rgb)
            # image_rgb.flags.writeable = True # Not strictly needed if not modifying image_rgb further for mediapipe
            # Frame is kept as BGR for cv2 drawing

            tracker_accuracy_feedback = "Tracker Accuracy: N/A"

            if results.pose_landmarks:
                all_landmarks = results.pose_landmarks.landmark

                # --- Tracker Accuracy Calculation ---
                visible_key_landmarks_count = 0
                for lm_idx in KEY_LANDMARK_INDICES: # Use the consolidated list of indices
                    if all_landmarks[lm_idx].visibility > VISIBILITY_THRESHOLD_FOR_ACCURACY:
                        visible_key_landmarks_count += 1
                
                if KEY_LANDMARK_INDICES:
                    accuracy_percentage = (visible_key_landmarks_count / len(KEY_LANDMARK_INDICES)) * 100
                    accuracy_status = "Poor"
                    if accuracy_percentage >= 85:
                        accuracy_status = "Good"
                    elif accuracy_percentage >= 60:
                        accuracy_status = "Fair"
                    tracker_accuracy_feedback = f"Tracker Accuracy: {accuracy_percentage:.0f}% - {accuracy_status}"
                else:
                    tracker_accuracy_feedback = "Tracker Accuracy: (No key LMs defined)"

                # Landmark drawing (UNCHANGED)
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=results.pose_landmarks,
                    connections=CUSTOM_SQUAT_CONNECTIONS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=connection_line_spec
                )
                for idx, landmark_data in enumerate(all_landmarks):
                    if idx in KEY_LANDMARK_INDICES: # Use the consolidated list of indices
                        if landmark_data.visibility > 0.5:
                            cx, cy = int(landmark_data.x * frame_width), int(landmark_data.y * frame_height)
                            cv2.circle(frame, (cx, cy), landmark_point_spec.circle_radius, landmark_point_spec.color, landmark_point_spec.thickness)
                
                form_feedback = get_feedback(all_landmarks)

                # Counter logic (UNCHANGED)
                if is_squat_down(all_landmarks) and squat_state == "up" and frame_count - last_counted_frame > min_frame_interval:
                    squat_counter += 1
                    squat_state = "down"
                    last_counted_frame = frame_count
                elif is_squat_up(all_landmarks) and squat_state == "down":
                    squat_state = "up"

                # --- Text Drawing ---
                cv2.putText(frame, f"Count: {squat_counter}", (20, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, WHITE_COLOR, 2, cv2.LINE_AA)
                
                # Simplified Feedback Text Color Logic
                if not form_feedback:
                    form_feedback_color = WHITE_COLOR 
                elif "Good Depth" in form_feedback or form_feedback == "Up":
                    form_feedback_color = GREEN_COLOR
                else: # All other non-empty feedbacks are corrective
                    form_feedback_color = RED_COLOR

                if form_feedback:
                    cv2.putText(frame, form_feedback, (20, 80),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, form_feedback_color, 2, cv2.LINE_AA)
                
                cv2.putText(frame, tracker_accuracy_feedback, (20, 120),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, YELLOW_COLOR, 2, cv2.LINE_AA)
            else:
                cv2.putText(frame, "No person detected", (20, 80), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, RED_COLOR, 2, cv2.LINE_AA)
                cv2.putText(frame, tracker_accuracy_feedback, (20, 120), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, YELLOW_COLOR, 2, cv2.LINE_AA)

            out.write(frame)

            if show_realtime:
                cv2.imshow('Squat Analysis - Realtime', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
    
    cap.release()
    out.release()
    if show_realtime:
        cv2.destroyAllWindows()
    print(f"Video with feedback saved as: {output_video}")

# File Directory

In [9]:
input_video_path = "squats.mp4" 
output_video_path = "squats_output.mp4" 

# Calling the video processing function

In [12]:
process_squat_video(input_video_path, output_video_path, show_realtime=True)


Video with feedback saved as: squats_output.mp4
